In [2]:
import pandas as pd
import numpy as np
import re
import pickle

In [3]:
df = pd.read_csv('data/data.csv', sep =';')

In [4]:
df['win_by'].value_counts()

Decision - Unanimous       1737
KO/TKO                     1647
Submission                 1083
Decision - Split            486
TKO - Doctor's Stoppage      70
Decision - Majority          56
Overturned                   35
DQ                           15
Could Not Continue           13
Other                         2
Name: win_by, dtype: int64

In [5]:
def winner (red,winner):
    if red == winner:
        return 1
    else:
        return 0

In [6]:
df['winner'] = df.apply(lambda x: winner(x['R_fighter'],x['Winner']),axis=1)

In [7]:
df['winner'].value_counts()
#whatever model gets selected needs to have an accuracy >67%

1    3470
0    1674
Name: winner, dtype: int64

In [8]:
#subset to decisions only
df = df[df['win_by'].str.contains("Decision")]

In [9]:
df['winner'].value_counts()

1    1456
0     823
Name: winner, dtype: int64

In [10]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'winner'],
      dtype='object')

In [11]:
#use the below to convert any string percents to actual percents
# df['col'] = df['col'].str.rstrip('%').astype('float') / 100.0
df['R_SIG_STR_pct'] = df['R_SIG_STR_pct'].str.rstrip('%').astype('float') / 100.0
df['B_SIG_STR_pct'] = df['B_SIG_STR_pct'].str.rstrip('%').astype('float') / 100.0
df['R_TD_pct'] = df['R_TD_pct'].str.rstrip('%').astype('float') / 100.0
df['B_TD_pct'] = df['B_TD_pct'].str.rstrip('%').astype('float') / 100.0


In [12]:
#subset to columns dealing with strikes, takedowns and submissions
X = pd.DataFrame()
X['strike_diff'] = df['R_SIG_STR_pct'] - df['B_SIG_STR_pct']
X['td_diff'] = df['R_TD_pct'] - df['B_TD_pct']
X['sub_diff'] = df['R_SUB_ATT'] - df['B_SUB_ATT']
X['pass_diff'] = df['R_PASS'] - df['B_PASS']

In [13]:
new1 = df["R_HEAD"].str.split("of", n = 1, expand = True) 
new2 = df["B_HEAD"].str.split("of", n = 1, expand = True) 
new3 = df["R_BODY"].str.split("of", n = 1, expand = True) 
new4 = df["B_BODY"].str.split("of", n = 1, expand = True) 
new5 = df["R_LEG"].str.split("of", n = 1, expand = True) 
new6 = df["B_LEG"].str.split("of", n = 1, expand = True) 
new7 = df["R_CLINCH"].str.split("of", n = 1, expand = True) 
new8 = df["B_CLINCH"].str.split("of", n = 1, expand = True) 
new11 = df["R_GROUND"].str.split("of", n = 1, expand = True) 
new12 = df["B_GROUND"].str.split("of", n = 1, expand = True) 

X['head_diff'] = new1[0].astype(float)/new1[1].astype(float) - new2[0].astype(float)/new2[1].astype(float)
X['body_diff'] = new3[0].astype(float)/new3[1].astype(float) - new4[0].astype(float)/new4[1].astype(float)
X['leg_dif'] = new5[0].astype(float)/new5[1].astype(float) - new6[0].astype(float)/new6[1].astype(float)
X['clinch_diff'] = new7[0].astype(float)/new7[1].astype(float) - new8[0].astype(float)/new8[1].astype(float)
X['ground_diff'] = new11[0].astype(float)/new11[1].astype(float) - new12[0].astype(float)/new12[1].astype(float)
X['kd_diff'] = df['R_KD'] - df['B_KD']


In [14]:
X = X.fillna(0)
X.describe()

,strike_diff,td_diff,sub_diff,pass_diff,head_diff,body_diff,leg_dif,clinch_diff,ground_diff,kd_diff
count,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000
mean,0.031435,0.068934,0.013602,0.594120,0.030515,0.027513,0.018602,0.016624,0.007799,-0.006143
std,0.169391,0.495057,1.280738,3.476849,0.181440,0.266089,0.267188,0.308276,0.270270,0.585872
min,-0.700000,-1.000000,-7.000000,-14.000000,-0.785714,-1.000000,-1.000000,-1.000000,-1.000000,-4.000000
25%,-0.080000,-0.270000,0.000000,-1.000000,-0.080367,-0.133333,-0.100000,-0.142857,0.000000,0.000000
50%,0.030000,0.000000,0.000000,0.000000,0.030780,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.140000,0.440000,0.000000,2.000000,0.140330,0.184772,0.138535,0.169478,0.000000,0.000000
max,0.800000,1.000000,10.000000,26.000000,0.739394,1.000000,1.000000,1.000000,1.000000,5.000000


In [15]:
X = X.iloc[:, [3,4,5,6]]
X.head()

,pass_diff,head_diff,body_diff,leg_dif
3,-1,0.048691,-0.059211,-0.054348
4,0,-0.228070,-0.017857,-0.173913
5,2,-0.128520,0.181698,0.571429
6,0,0.215270,0.040026,-0.184982
7,0,-0.022640,0.100791,-0.061086


In [16]:
#initialize scale test and train data sets separetly to avoid leakage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_scaler=StandardScaler()
test_scaler=StandardScaler()

In [17]:
#set up dependant variable
y = df['winner'].copy()
len(y)

2279

# ADD A COMBINED DATA SET TO DETERMINE CORRELATION FOR FEATURE SELECTION!


In [18]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=21)

In [19]:
# Create a scaler 
train_scaler.fit(X_train)
test_scaler.fit(X_test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
#archive the training scaler
filename = open('data/fitted_scaler.pkl', 'wb')
pickle.dump(train_scaler, filename)
filename.close()

In [21]:
# Now transform
X_train_scaled = train_scaler.transform(X_train)
X_test_scaled = test_scaler.transform(X_test)

In [22]:
# confirm that that did what we wanted
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

In [23]:
X_train_scaled_df.head()

,pass_diff,head_diff,body_diff,leg_dif
0,0.990243,-0.004732,0.413550,-0.240265
1,-0.736032,0.012672,3.642676,-0.051557
2,2.141093,0.126465,-1.496516,0.806206
3,-0.160607,0.067888,1.889305,0.074248
4,-0.448320,-0.469773,-1.325210,3.093573


In [22]:
# Pickle training data set for future graphical work
#X_train.to_pickle('data/train.pkl') 

In [23]:
#random forest, logistic regression, Naive-Bayes? 
#Reminder: target accuracy is >67%
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB

from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_curve
from sklearn.metrics import roc_auc_score

lr = LogisticRegression() 
rf = RandomForestClassifier() 
gnb = GaussianNB()

lr.fit(X_train_scaled_df,y_train)
rf.fit(X_train_scaled_df,y_train)
gnb.fit(X_train_scaled_df,y_train)

lr_pred = lr.predict(X_test_scaled_df)
rf_pred = rf.predict(X_test_scaled_df)
gnb_pred = gnb.predict(X_test_scaled_df)

auc_rf=metrics.roc_auc_score(y_test, rf_pred)
acc_rf = metrics.accuracy_score(y_test, rf_pred)
f1_rf = metrics.f1_score(y_test, rf_pred)

auc_lr=metrics.roc_auc_score(y_test, lr_pred)
acc_lr = metrics.accuracy_score(y_test, lr_pred)
f1_lr = metrics.f1_score(y_test, lr_pred)

auc_nb=metrics.roc_auc_score(y_test, gnb_pred)
acc_nb = metrics.accuracy_score(y_test, gnb_pred)
f1_nb = metrics.f1_score(y_test, gnb_pred)

# Display
print('Naive-Bayes:')
print('F1 Score', "%.4f" % round(f1_nb,4))
print('Accuracy', "%.4f" % round(acc_nb,4))
print('AUC Score', "%.4f" % round(auc_nb,4))
print('')
print('Random Forrest:')
print('F1 Score', "%.4f" % round(f1_rf,4))
print('Accuracy', "%.4f" % round(acc_rf,4))
print('AUC Score', "%.4f" % round(auc_rf,4))
print('')
print('Logistic Regression:')
print('F1 Score', "%.4f" % round(f1_lr,4))
print('Accuracy', "%.4f" % round(acc_lr,4))
print('AUC Score', "%.4f" % round(auc_lr,4))
print('')
print('All three models exceed the target accuracy of 67%.')

Naive-Bayes:
F1 Score 0.8377
Accuracy 0.7825
AUC Score 0.7508

Random Forrest:
F1 Score 0.7890
Accuracy 0.7316
AUC Score 0.7187

Logistic Regression:
F1 Score 0.8446
Accuracy 0.7895
AUC Score 0.7536

All three models exceed the target accuracy of 67%.


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
#archive the ml models
filename = open('data/rf.pkl', 'wb')
pickle.dump(rf, filename)
filename.close()

filename = open('data/lr.pkl', 'wb')
pickle.dump(lr, filename)
filename.close()

filename = open('data/gnb.pkl', 'wb')
pickle.dump(gnb, filename)
filename.close()

In [24]:
# create lists from the metrics we produced.
f1=[f1_nb, f1_lr, f1_rf]
acc=[acc_nb, acc_lr, acc_rf]
auc=[auc_nb, auc_lr, auc_rf]
# Define a function that will round our metrics.
def rounder(metric):
    scores_list=[]
    for score in metric:
        scores_list.append(round(float(score*100),1))
    return scores_list
# Apply it to each of the three lists.
f1_scores=rounder(f1)
acc_scores=rounder(acc)
auc_scores=rounder(auc)
score_types=['F1 score', 'Accuracy', 'AUC score']

In [25]:
# Comparison of model metrics
models=['Naive-Bayes', 'Logistic Regression', 'Random Forest']
index=['F1 score', 'Accuracy', 'AUC score']
compare_models=pd.DataFrame([f1_scores, acc_scores, auc_scores], index=index, columns=models)
# save to pickle, for later use by plotly dash app.
compare_models.to_pickle('data/compare_models.pkl')

In [26]:
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


# Let's display that with plotly.
mydata1 = go.Bar(
    x=compare_models.loc['F1 score'].index,
    y=compare_models.loc['F1 score'],
    name=compare_models.index[0],
    marker=dict(color='#332225')
)
mydata2 = go.Bar(
    x=compare_models.loc['Accuracy'].index,
    y=compare_models.loc['Accuracy'],
    name=compare_models.index[1],
    marker=dict(color='#24434C')
)
mydata3 = go.Bar(
    x=compare_models.loc['AUC score'].index,
    y=compare_models.loc['AUC score'],
    name=compare_models.index[2],
    marker=dict(color='#4c0000')
)
mylayout = go.Layout(
    title='Comparison of Judges Accuracy',
    xaxis = dict(title = 'Possible Judges'), # x-axis label
    yaxis = dict(title = 'Score'), # y-axis label
    
)
fig = go.Figure(data=[mydata1, mydata2, mydata3], layout=mylayout)
iplot(fig)

In [27]:
X_test_scaled_df.describe()

,pass_diff,head_diff,body_diff,leg_dif
count,5.700000e+02,5.700000e+02,5.700000e+02,5.700000e+02
mean,4.148728e-16,1.236827e-17,3.895519e-19,-2.648953e-17
std,1.000878e+00,1.000878e+00,1.000878e+00,1.000878e+00
min,-3.079599e+00,-4.261620e+00,-3.264908e+00,-3.785501e+00
25%,-4.897067e-01,-5.740583e-01,-5.604996e-01,-5.079976e-01
50%,-2.019409e-01,7.947149e-03,-1.106372e-01,-1.224090e-01
75%,3.735906e-01,5.915669e-01,6.079632e-01,4.881063e-01
max,7.279969e+00,2.817825e+00,3.491101e+00,3.540683e+00


In [28]:
new_data = [[-1.640770,0.593508, 1.375416,1.098622]]
print(lr.predict(new_data))
print(rf.predict(new_data))
print(gnb.predict(new_data))

[0]
[0]
[1]
